# Dashboarding SF Data

In [33]:
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [34]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)
type(map_box_api)

str

In [35]:

# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

In [36]:
housing_unit_by_year = sfo_data.groupby('year')
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_unit = housing_unit_by_year['housing_units'].first()
    boundry = housing_unit.std()
    return housing_unit.hvplot.bar(
        x="year",
        y="housing_units",
        xlabel='Year',
        ylabel='Housing Units',
        label=f"Housing Units in San Francisco from {housing_unit.index.min()} to {housing_unit.index.max()}",
        ylim=(housing_unit.min()-boundry,housing_unit.max()+boundry)
    ).opts(yformatter="%.0f")


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    return housing_unit_by_year['gross_rent'].mean().hvplot(
        label='Average Gross Rent in San Francisco',
        xlabel='Year', ylabel='Gross Rent',
        xlim=(housing_unit_by_year.first().index.min(), housing_unit_by_year.first().index.max())
    )


def average_sales_price():
    """Average Sales Price Per Year."""
    
    return housing_unit_by_year['sale_price_sqr_foot'].mean().hvplot(
        label='Average Sales Price in San Francisco',
        xlabel='Year', ylabel='Avg. Sale Price',
        xlim=(housing_unit_by_year.first().index.min(), housing_unit_by_year.first().index.max())
    )


neighborhood_df = sfo_data.groupby(['year','neighborhood']).mean().reset_index()

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    return neighborhood_df.hvplot(
        label='Average Price per Square feet by year',
        x='year',
        y='sale_price_sqr_foot',
        xlabel='Year',
        ylabel='Avg. Sales Price per Square Foot', 
        #shared_axes=False,
        #ylim=(top_expensive_neighborhood['sale_price_sqr_foot'].min()-100,top_expensive_neighborhood['sale_price_sqr_foot'].max()+100),
        groupby='neighborhood'
    )

avg_values_neighborhood = neighborhood_df.drop(columns=['year']).groupby('neighborhood').mean().round(2)
top_expensive_neighborhood = avg_values_neighborhood.sort_values("sale_price_sqr_foot", ascending = False).head(10)

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    return top_expensive_neighborhood['sale_price_sqr_foot'].hvplot.bar(
        label='Top 10 Expensive Neighborhoods in SFO',
        xlabel='Neighborhood',
        shared_axes=False,
        ylabel='Avg. Sales Price per Square Foot', 
        rot=45
    )


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    return px.parallel_coordinates(
        top_expensive_neighborhood, 
        title='Parallel Coordinates Plot',
        color='sale_price_sqr_foot'
    )


def parallel_categories():
    """Parallel Categories Plot."""
    
    return px.parallel_categories(
        top_expensive_neighborhood.reset_index(), 
        color='sale_price_sqr_foot',
        title='Parallel Categories Plot',
        color_continuous_scale=px.colors.sequential.Agsunset
    )


def neighborhood_map():
    """Neighborhood Map"""
    
    new_avg_values_neighborhood=avg_values_neighborhood.reset_index().rename(columns = {'neighborhood':'Neighborhood'})
    neighborhood_location=pd.merge(df_neighborhood_locations,new_avg_values_neighborhood, on='Neighborhood')
    
    return px.scatter_mapbox(neighborhood_location,
                             lat="Lat",
                             lon="Lon",
                             size='sale_price_sqr_foot',
                             color='gross_rent',
                             hover_name='Neighborhood',
                             zoom=11,
                             color_continuous_scale=px.colors.sequential.thermal
                            )

In [37]:
location_column = pn.Column(' Visual analysis of the real estate investment opportunities in San Fransco with interactive map with the average prices per neighborhood',
                           neighborhood_map())

yearly_market_analysis = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())

neighborhood_analysis = pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods())

parallel_plot_analysis = pn.Column(parallel_coordinates(), parallel_categories())


panel_content = pn.Tabs(
    ('Welcome', location_column),
    ('Yearly Market Analysis', yearly_market_analysis),
    ('Neighborhood Analysis', neighborhood_analysis),
    ('Parallel Plot Analysis', parallel_plot_analysis)
)

panel = pn.Column(f'# Real Estate Analysis of San Francisco from {sfo_data.index.min()} to {sfo_data.index.max()}<br/>',
                  panel_content
                 )

In [38]:
panel.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Curve)
            [2] HoloViews(Curve)
        [2] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] HoloViews(Bars)
        [3] Column
            [0] Plotly(Figure)
            [1] Plotly(Figure)

In [40]:
panel.show()

Launching server at http://localhost:53727
